# More tests...

In [1]:
%matplotlib qt
import os
import sys
import time

import numpy as np
import scipy.optimize
import sklearn.cluster
import matplotlib.pyplot as plt
import skimage.feature
import skimage.filters
import skimage.morphology

import hyperspy.api as hs

sys.path.append("../")
import windows
import methods

In [2]:
DATA_DIRECTORY = "../data"
data_filenames = []

# Make a list of filenames
for dataset_directory in os.listdir(DATA_DIRECTORY):
    dataset_directory_path = os.path.join(DATA_DIRECTORY, dataset_directory)
    data_filenames.append({
        "dataset_name": dataset_directory,
        "dataset_file_list": [
            os.path.join(dataset_directory_path, filename) 
            for filename in os.listdir(dataset_directory_path) 
            if filename[-4:] == ".dm3" or filename[-4:] == ".emd"
        ]
    })

## Data loading

In [3]:
s = hs.load(data_filenames[0]["dataset_file_list"][7])

## Initial peak finding

In [4]:
peaks_list = skimage.feature.peak_local_max(s.data[:], min_distance=100, threshold_abs=100, exclude_border=True, num_peaks=12)

In [5]:
spot_fwhm = methods.get_gaussian_fwhm(s.data, peaks_list[0])

In [6]:
selected_radius = spot_fwhm * 2
inner_background_radius = spot_fwhm * 5
outer_background_radius = spot_fwhm * 6

## Peak refinement
Refine the peaks by identifying spurrious peaks. Refine by grouping by radial distance from an estimated central point, and by asimuthal spread.

### Radial Grouping

In [7]:
central_spot_estimate = np.mean(peaks_list, axis=0)

distances_to_central_spot = np.linalg.norm([peak_pos - central_spot_estimate for peak_pos in peaks_list], axis=1)
distances_to_central_spot = np.expand_dims(distances_to_central_spot, axis=1)

position_distance = np.concatenate((peaks_list, distances_to_central_spot), axis=1)
position_distance = position_distance[position_distance[:, 2].argsort()]

radial_distance_steps = np.diff(position_distance[:, 2])
radial_distance_steps_median = np.median(radial_distance_steps)
radial_distance_steps_normalised = radial_distance_steps / radial_distance_steps_median

In [8]:
big_step_indices = []
for index, normalised_radial_step in enumerate(radial_distance_steps_normalised):
    if normalised_radial_step > 5:
        big_step_indices.append(index+1)

In [9]:
groups = []
prev_step_inx = 0
for step_index in big_step_indices:
    groups.append(position_distance[prev_step_inx:step_index, :2])
    prev_step_inx = step_index
groups.append(position_distance[step_index:, :2])

In [10]:
fig, ax = plt.subplots()
ax.imshow(s.data, vmin=0, vmax=200)
group_colors = 'r', 'g', 'b'
for group_color, group in zip(group_colors, groups):
    color = group_color
    for peak_point in group:
        circle = plt.Circle(peak_point[::-1], outer_background_radius*1.1, color=color, fill=False)
        ax.add_patch(circle)
central_spot_patch = plt.Circle(central_spot_estimate[::-1], outer_background_radius, color='r', fill=True)
ax.add_patch(central_spot_patch)
if any([len(group)!=6 for group in groups]):
    ax.set_title("Error!")
plt.show()

### Central Spot Refinement

Tried a trilateration technique, but got multiple solutions. Then found this circumcentre thing. Will now take a set of circumcentres...

In [71]:
test_points = groups[0][:3]

In [84]:
circumcentre_triplets = []
indices = np.arange(len(groups[0]))
for _ in range(6):
    circumcentre_triplets.append([groups[0][i] for i in indices[:3]])
    print(indices[:3])
    indices = np.roll(indices, shift=1)

[0 1 2]
[5 0 1]
[4 5 0]
[3 4 5]
[2 3 4]
[1 2 3]


In [89]:
def get_circumcentre(points):
    """Calculates the circumcentre (equidistant point) from 3 other points. Thus 'points' should be a 3x2. See https://www.wikiwand.com/en/Circumscribed_circle#/Circumcircle_equations"""
    A, B, C = points
    Ax, Ay = A; Bx, By = B; Cx, Cy = C
    D = 2*(Ax*(By-Cy)+Bx*(Cy-Ay)+Cx*(Ay-By))
    Ux = ( (Ax**2+Ay**2)*(By-Cy) + (Bx**2+By**2)*(Cy-Ay) + (Cx**2+Cy**2)*(Ay-By) ) / D
    Uy = ( (Ax**2+Ay**2)*(Cx-Bx) + (Bx**2+By**2)*(Ax-Cx) + (Cx**2+Cy**2)*(Bx-Ax) ) / D
    return np.array((Ux, Uy))

circs = np.array([get_circumcentre(triplet) for triplet in circumcentre_triplets])

(6, 2)

In [91]:
median_circ = np.median(circs, axis=0)

(2,)

In [92]:
fig, ax = plt.subplots()
ax.imshow(s.data, vmin=0, vmax=200)
for test_point in test_points:
    circle = plt.Circle(test_point[::-1], outer_background_radius*1, color='r', fill=True)
    ax.add_patch(circle)
circle = plt.Circle(median_circ[::-1], outer_background_radius*0.2, color='r', fill=True)
ax.add_patch(circle)
plt.show()

## Intensity Measurement

In [ ]:
peak_intensities = []
image_window = windows.CircularWindow(selected_radius, inner_background_radius, outer_background_radius, s_dilated)
for peak_point in peaks_list:
    peak_intensities.append(image_window.get_integrated_intensity(*peak_point))